In [21]:
# imports
import pandas as pd

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# cnn imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers.embeddings import Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [22]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[26, 9], [48, 15], [34, 49], [36, 15], [19], [13], [3, 49], [44, 48], [3, 15], [43, 7, 9, 37]]
[[26  9  0  0]
 [48 15  0  0]
 [34 49  0  0]
 [36 15  0  0]
 [19  0  0  0]
 [13  0  0  0]
 [ 3 49  0  0]
 [44 48  0  0]
 [ 3 15  0  0]
 [43  7  9 37]]


In [23]:
num_steps = 8
hidden_size = 8

model = Sequential()
#model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocab_size)))
model.add(Activation('softmax'))

In [24]:
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 8, 8)              400       
_________________________________________________________________
lstm_11 (LSTM)               (None, 8, 8)              544       
_________________________________________________________________
lstm_12 (LSTM)               (None, 8, 8)              544       
_________________________________________________________________
dropout_4 (Dropout)          (None, 8, 8)              0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 8, 50)             450       
_________________________________________________________________
activation_1 (Activation)    (None, 8, 50)             0         
Total params: 1,938
Trainable params: 1,938
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
logits = [0.0, 1.0, -1.0]
exps = [np.exp(i) for i in logits]
sum_of_exps = sum(exps)
softmax = [j/sum_of_exps for j in exps]


In [26]:
a = np.array([[3,1,3],[2,-2,6],[8,5,9]]) 
b = np.array([softmax[0],softmax[1],softmax[2]]) 
np.dot(a,b)


array([ 1.66951809, -0.30084153,  6.09430771])